In [1]:
%load_ext autoreload
%autoreload 2
from commons import *

In [2]:
# Guitarlele tuning
# guitar = Guitar(tuning=['A2','D3','G3','C3','E4','A5'])
guitar = Guitar()
guitar.printFretboardDiagram()

 E  █ F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   |
 B  █ C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  |
 G  █ G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   |
 D  █ D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   |
 A  █ A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   |
 E  █ F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   |
                              •           •           •                 ••                •           •           •   


In [67]:
natural_scale   = [0, 2, 4, 5, 7, 9, 11]
# minor_melodic   = [0, 2, 3, 5, 7, 9, 10]
# minor_harmonic  = [0, 2, 3, 5, 7, 8, 11]
# major_harmonic  = [0, 2, 3, 5, 7, 8, 11]

def parseScaleListWH(notesList):
    res = [0]
    for i in notesList:
        halfs = i.count('H')
        fulls = i.count('W')
        res.append(res[-1] + halfs + fulls*2)
    return res

def parseScaleList(notesList, reference):
    if 'W' in notesList[0] or 'H' in notesList[0]:
        return parseScaleListWH(notesList)

    res = []
    for i in notesList:
        i = i.replace(' ','')
        flats = i.count('b')
        sharps = i.count('#')
        num = int(i.replace('b','').replace('#','')) - 1
        res.append(reference[num] - flats + sharps)
    res.sort()
    return res

class Scale:
    def __init__(self, progression, name=None, reference = natural_scale): #reference is major
        if isinstance(progression, list):
            if isinstance(progression[0], int):
                self.prog = list(progression)
            elif isinstance(progression[0], str):
                self.prog = parseScaleList(progression, reference)
            else:
                raise Exception(f"Unknown dtype for progression[0]: {type(progression[0])}")
        elif isinstance(progression, str):
            self.prog = parseScaleList(progression.split(','), reference)
        else:
            raise Exception(f"Unknown dtype for progression: {type(progression)}")
        self.name = name
        self.aliases = []

    def addAlias(self, alias):
        self.aliases.append(alias)

    def __str__(self):
        # prog = [f"{i:>3}" for i in self.progression]
        name = self.name or "Unnamed"
        return f"{name:15}: {self.prog}"

    def __getitem__(self, item):
        return self.prog[item]
    
    def __len__(self):
        return len(self.prog)

    def mode(self, num=1, name=None):
        assert num > 0 and num <= len(self.prog)
        num -= 1
        root = self.prog[num]
        
        prog = [i - root for i in self.prog[num:]] \
            + [i - root + 12 for i in self.prog[:num]]
        # print(prog)
        return Scale(prog, name=name)
    
    def distance(self, other):
        assert len(self) == len(other)
        dist = 0
        for i in range(len(self)):
            if self[i] != other[i]:
                dist += 1
        return dist

    def obtainRelativeName(self, other):
        assert len(self) == len(other)
        mods = []
        for i in range(len(self)):
            if self[i] != other[i]:
                dist = self[i] - other[i]
                if dist < 0:
                    mod = 'b' * (-dist)
                # elif dist == 0:
                #     mod = '♮'
                else:
                    mod = '#' * dist
                mods.append(f"{mod}{i+1}")
        return other.name + " " + " ".join(mods)


print(Scale('1,2#,3,4,5,6'))
print(Scale('WH,H,H,W,W'))

Unnamed        : [0, 3, 4, 5, 7, 9]
Unnamed        : [0, 3, 4, 5, 7, 9]


In [70]:
scales = []

# major = Scale([0, 2, 4, 5, 7, 9, 11], name='major')
major = Scale(['W', 'W', 'H', 'W', 'W', 'W'], name='ionian')
major.addAlias('major')
dorian = major.mode(2, name="dorian")
phrygian = major.mode(3, name="phrygian")
lydian = major.mode(4, name="lydian")
mixolydian = major.mode(5, name="mixolydian")
aeolian = major.mode(6, name="aeolian")
aeolian.addAlias('natural minor')
locrian = major.mode(7, name='locrian')

major_modes = [
    major,
    dorian,
    phrygian,
    lydian,
    mixolydian,
    aeolian,
    locrian
]

print(NoteSet(major, Note('C3')))
print(NoteSet(dorian, Note('D3')))
print(NoteSet(phrygian, Note('E3')))
print(NoteSet(lydian, Note('F3')))
print(NoteSet(mixolydian, Note('G3')))
print(NoteSet(aeolian, Note('A3')))
print(NoteSet(locrian, Note('B3')))


C3  D3  E3  F3  G3  A3  B3  
D3  E3  F3  G3  A3  B3  C4  
E3  F3  G3  A3  B3  C4  D4  
F3  G3  A3  B3  C4  D4  E4  
G3  A3  B3  C4  D4  E4  F4  
A3  B3  C4  D4  E4  F4  G4  
B3  C4  D4  E4  F4  G4  A4  


In [ ]:

# scales['melodic minor'] = [0, 2, 3, 5, 7, 9, 10]
# scales['dorian b2'] = mode(scales['melodic minor'], 2)
# scales['lydian augmented'] = mode(scales['melodic minor'], 3)
# scales['lydian dominant'] = mode(scales['melodic minor'], 4)
# scales['aeolian dominant'] = mode(scales['melodic minor'], 5)
# scales['half diminished'] = mode(scales['melodic minor'], 6)
# scales['altered'] = mode(scales['melodic minor'], 7)
# print(NoteSet(scales['melodic minor'], Note('C3')))
# print(NoteSet(scales['dorian b2'], Note('D3')))
# print(NoteSet(scales['lydian augmented'], Note('D#3')))
# print(NoteSet(scales['lydian dominant'], Note('F3')))
# print(NoteSet(scales['aeolian dominant'], Note('G3')))
# print(NoteSet(scales['half diminished'], Note('A4')))
# print(NoteSet(scales['altered'], Note('A#4')))

In [71]:


mel_minor = Scale(['1', '2', 'b3', '4', '5', '6', '7'])
mel_minor.addAlias('ascending melodic minor')

mel_minor_modes = [mel_minor.mode(i) for i in range(2, 8)]


mel_minor_noteset = NoteSet(mel_minor, Note('C3'))
print(mel_minor_noteset)
for index, note in enumerate(mel_minor_noteset[1:]):
    print(NoteSet(mel_minor_modes[index], note))

mel_minor_modes.insert(0, mel_minor)
for i in mel_minor_modes:
    min_dist = 1000
    chosen = None
    for j in major_modes:
        dist = i.distance(j)
        if dist < min_dist:
            chosen = [j]
            min_dist = dist
        elif dist == min_dist:
            chosen.append(j)
    # print(f"Closest to [{i}]: {[str(i) for i in chosen]}")
    # print(f"Dist: {min_dist}")
    names = [i.obtainRelativeName(j) for j in chosen]
    print(names)
    

    

C3  D3  D#3 F3  G3  A3  B3  
D3  D#3 F3  G3  A3  B3  C4  
D#3 F3  G3  A3  B3  C4  D4  
F3  G3  A3  B3  C4  D4  D#4 
G3  A3  B3  C4  D4  D#4 F4  
A3  B3  C4  D4  D#4 F4  G4  
B3  C4  D4  D#4 F4  G4  A4  
['ionian b3', 'dorian #7']
['dorian b2', 'phrygian #6']
['lydian #5']
['lydian b7', 'mixolydian #4']
['mixolydian b6', 'aeolian #3']
['aeolian b5', 'locrian #2']
['locrian b4']


In [72]:
# scales['harmonic minor'] = [0, 2, 3, 5, 7, 8, 11]


har_minor = Scale([0, 2, 3, 5, 7, 8, 11])
har_minor.addAlias('melodic minor')

har_minor_modes = [har_minor.mode(i) for i in range(2, 8)]

har_minor_noteset = NoteSet(har_minor, Note('C3'))
print(har_minor_noteset)
for index, note in enumerate(har_minor_noteset[1:]):
    print(NoteSet(har_minor_modes[index], note))

har_minor_modes.insert(0, har_minor)
for i in har_minor_modes:
    min_dist = 1000
    chosen = None
    for j in major_modes:
        dist = i.distance(j)
        if dist < min_dist:
            chosen = [j]
            min_dist = dist
        elif dist == min_dist:
            chosen.append(j)
    # print(f"Closest to [{i}]: {[str(i) for i in chosen]}")
    # print(f"Dist: {min_dist}")
    names = [i.obtainRelativeName(j) for j in chosen]
    print(i, ":", names)
    

    

# Scale(['1', '2', 'b3', '4', '5', 'b6', '7'])
# scales['locrian nat6'] = mode(scales['harmonic minor'], 2)
# scales['major #5'] = mode(scales['harmonic minor'], 3)
# scales['dorian #4'] = mode(scales['harmonic minor'], 4)
# scales['phrygian dominant'] = mode(scales['harmonic minor'], 5)
# scales['lydian #2'] = mode(scales['harmonic minor'], 6)
# scales['altered dominant bb7'] = mode(scales['harmonic minor'], 7)
# print(NoteSet(scales['harmonic minor'], Note('C3')))
# print(NoteSet(scales['locrian nat6'], Note('D3')))
# print(NoteSet(scales['major #5'], Note('D#3')))
# print(NoteSet(scales['dorian #4'], Note('F3')))
# print(NoteSet(scales['phrygian dominant'], Note('G3')))
# print(NoteSet(scales['lydian #2'], Note('G#3')))
# print(NoteSet(scales['altered dominant bb7'], Note('B4')))

C3  D3  D#3 F3  G3  G#3 B3  
D3  D#3 F3  G3  G#3 B3  C4  
D#3 F3  G3  G#3 B3  C4  D4  
F3  G3  G#3 B3  C4  D4  D#4 
G3  G#3 B3  C4  D4  D#4 F4  
G#3 B3  C4  D4  D#4 F4  G4  
B3  C4  D4  D#4 F4  G4  G#4 
Unnamed        : [0, 2, 3, 5, 7, 8, 11] : ['aeolian #7']
Unnamed        : [0, 1, 3, 5, 6, 9, 10] : ['locrian #6']
Unnamed        : [0, 2, 4, 5, 8, 9, 11] : ['ionian #5']
Unnamed        : [0, 2, 3, 6, 7, 9, 10] : ['dorian #4']
Unnamed        : [0, 1, 4, 5, 7, 8, 10] : ['phrygian #3']
Unnamed        : [0, 3, 4, 6, 7, 9, 11] : ['lydian #2']
Unnamed        : [0, 1, 3, 4, 6, 8, 9] : ['locrian b4 b7']


In [72]:
scales['harmonic major'] = [0, 2, 4, 5, 7, 8, 11]
scales['dorian b5'] = mode(scales['harmonic major'], 2)
scales['phrygian b4'] = mode(scales['harmonic major'], 3)
scales['lydian b4'] = mode(scales['harmonic major'], 4)
scales['mixolydian b2'] = mode(scales['harmonic major'], 5)
scales['lydian augmented #2'] = mode(scales['harmonic major'], 6)
scales['locrian bb7'] = mode(scales['harmonic major'], 7)


print(NoteSet(scales['harmonic major'], Note('C3')))
print(NoteSet(scales['dorian b5'], Note('D3')))
print(NoteSet(scales['phrygian b4'], Note('E3')))
print(NoteSet(scales['lydian b4'], Note('F3')))
print(NoteSet(scales['mixolydian b2'], Note('G3')))
print(NoteSet(scales['lydian augmented #2'], Note('G#3')))
print(NoteSet(scales['locrian bb7'], Note('B4')))

C D E F G G# B
D E F G G# B C
E F G G# B C D
F G G# B C D E
G G# B C D E F
G# B C D E F G
B C D E F G G#


In [57]:
scales['diminished'] = [0, 2, 3, 5, 6, 8, 9, 11]
scales['inverse diminished'] = mode(scales['diminished'], 2)

print(NoteSet(scales['diminished'], Note('C3')))
print(NoteSet(scales['inverse diminished'], Note('D3')))
print(NoteSet(scales['diminished'], Note('D#3')))
print(NoteSet(scales['inverse diminished'], Note('F3')))
print(NoteSet(scales['diminished'], Note('F#3')))
print(NoteSet(scales['inverse diminished'], Note('G#3')))
print(NoteSet(scales['diminished'], Note('A4')))
print(NoteSet(scales['inverse diminished'], Note('B4')))

C D D# F F# G# A B
D D# F F# G# A B C
D# F F# G# A B C D
F F# G# A B C D D#
F# G# A B C D D# F
G# A B C D D# F F#
A B C D D# F F# G#
B C D D# F F# G# A


In [56]:
scales['augmented'] = [0, 3, 4, 7, 8, 11]
scales['inverse augmented'] = mode(scales['augmented'], 2)

print(NoteSet(scales['augmented'], Note('C3')))
print(NoteSet(scales['inverse augmented'], Note('D#3')))
print(NoteSet(scales['augmented'], Note('E3')))
print(NoteSet(scales['inverse augmented'], Note('G3')))
print(NoteSet(scales['augmented'], Note('G#3')))
print(NoteSet(scales['inverse augmented'], Note('B4')))

C D# E G G# B
D# E G G# B C
E G G# B C D#
G G# B C D# E
G# B C D# E G
B C D# E G G#
